In [1]:
import pandas as pd
import hopsworks
import joblib
import datetime
import dataframe_image as df1
from datetime import datetime
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/39062
Connected. Call `.close()` to terminate connection gracefully.


In [2]:
mr = project.get_model_registry()
model = mr.get_model('bank_note', version=1)
model_dir = model.download()
model = joblib.load(model_dir + '/bank_note_model.pkl')

Connected. Call `.close()` to terminate connection gracefully.

In [3]:
feature_view = fs.get_feature_view(name='bank_notes_dataframe', version=1)

In [4]:
batch_data = feature_view.get_batch_data()
predictions = model.predict(batch_data)

2023-05-04 16:28:18,492 INFO: USE `bwhiz_featurestore`
2023-05-04 16:28:19,455 INFO: SELECT `fg0`.`variance` `variance`, `fg0`.`skewness` `skewness`, `fg0`.`curtosis` `curtosis`, `fg0`.`entropy` `entropy`
FROM `bwhiz_featurestore`.`bank_notes_dataframe_1` `fg0`


In [5]:
predictions

array([1, 1, 1, ..., 0, 0, 0])

In [6]:
batch_data

,variance,skewness,curtosis,entropy
0,0.22432,-0.52147,-0.40386,1.20170
1,-2.29870,-5.22700,5.63000,0.91722
2,-4.50460,-5.81260,10.88670,-0.52846
3,3.96630,10.16840,-4.11310,-4.60560
4,-2.28040,-0.30626,1.33470,1.37630
...,...,...,...,...
1343,3.77580,7.17830,-1.51950,0.40128
1344,4.60540,-4.07650,2.75870,0.31981
1345,0.37980,0.70980,0.75720,-0.44440
1346,2.46730,1.39260,1.71250,0.41421


In [7]:
# prediciton on the last bank note sent :
latest_bank_note = predictions[predictions.size - 1]

In [8]:
bank_note_fg = fs.get_feature_group(name='bank_notes_dataframe', version=1)
df = bank_note_fg.read()
df

2023-05-04 16:28:25,161 INFO: USE `bwhiz_featurestore`
2023-05-04 16:28:25,954 INFO: SELECT `fg0`.`variance` `variance`, `fg0`.`skewness` `skewness`, `fg0`.`curtosis` `curtosis`, `fg0`.`entropy` `entropy`, `fg0`.`output` `output`
FROM `bwhiz_featurestore`.`bank_notes_dataframe_1` `fg0`


,variance,skewness,curtosis,entropy,output
0,0.22432,-0.52147,-0.40386,1.20170,1
1,-2.29870,-5.22700,5.63000,0.91722,1
2,-4.50460,-5.81260,10.88670,-0.52846,1
3,3.96630,10.16840,-4.11310,-4.60560,0
4,-2.28040,-0.30626,1.33470,1.37630,1
...,...,...,...,...,...
1343,3.77580,7.17830,-1.51950,0.40128,0
1344,4.60540,-4.07650,2.75870,0.31981,0
1345,0.37980,0.70980,0.75720,-0.44440,0
1346,2.46730,1.39260,1.71250,0.41421,0


In [9]:
### getting the actual label of the last row of data:
label = df.iloc[-1]['output']
label

0.0

In [10]:
monitor_fg = fs.get_or_create_feature_group(name = 'bank_note_predictions',
                                           version = 1,
                                           primary_key = ['datetime'],
                                           description = 'bank note prediction/output Monitoring')

In [11]:
now = datetime.now().strftime("%m/%d/%y, %H:%M:%S")

data = {
    'prediction' : [latest_bank_note],
    'label' : [label],
    'datetime' :[now]
}

monitor_df = pd.DataFrame(data)
monitor_fg.insert(monitor_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/39062/fs/38982/fg/42692


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/39062/jobs/named/bank_note_predictions_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x1c8ddfda1c0>, None)

In [12]:
history_df = monitor_fg.read()
history_df

2023-05-04 16:29:34,932 INFO: USE `bwhiz_featurestore`
2023-05-04 16:29:35,688 INFO: SELECT `fg0`.`prediction` `prediction`, `fg0`.`label` `label`, `fg0`.`datetime` `datetime`
FROM `bwhiz_featurestore`.`bank_note_predictions_1` `fg0`


,prediction,label,datetime
0,0,0.0,"05/04/23, 16:28:29"


In [13]:
df_recent = history_df.tail(5)

In [14]:
df1.export(df_recent, 'assets/df_recent.png') #save an image of the dataframe's last five output

In [16]:
predictions = history_df['prediction']
labels = history_df['label']

results = confusion_matrix(labels, predictions)
print(results)

[[1]]


In [ ]:
try:
    if results.shape == (3,3):
        cm = sns.heatmap(results, annot=True)
        
        fig = cm.get_figure()
        fig.savefig('assets/confusion_matrix.png')
        results
    else:
        print('The confusion is isnt a square matrix ')